## 1. Which drivers consistently improve from qualifying to race results, and how often do they gain or lose positions?
### This helps determine how much a driver's starting position influences their final placement.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.driver_improvement AS
SELECT
    q.Driver_ID,
    d.First_Name,
    d.Last_Name,
    q.Year,
    COUNT(*) AS total_races,
    ROUND(AVG(q.Position), 2) AS avg_qualifying_position,
    ROUND(AVG(r.Position), 2) AS avg_race_position,
    ROUND(AVG(q.Position - r.Position), 2) AS avg_positions_gained,
    COUNT(CASE WHEN (q.Position - r.Position) > 0 THEN 1 END) AS races_improved,
    COUNT(CASE WHEN (q.Position - r.Position) < 0 THEN 1 END) AS races_lost_positions
FROM saffatandsourik.formula1_int.Qualifying_Results q
JOIN saffatandsourik.formula1_int.Race_Results r
    ON q.Driver_ID = r.Driver_ID
    AND q.Year = r.Season
    AND q.Round = r.Round
JOIN saffatandsourik.formula1_int.Drivers d
    ON q.Driver_ID = d.Driver_ID
GROUP BY q.Driver_ID, d.First_Name, d.Last_Name, q.Year
ORDER BY q.Year DESC, avg_positions_gained DESC;


Query is running:   0%|          |

""


## 2. Which constructors consistently perform well across seasons, and how do their race wins and podium finishes compare?
### This question evaluates which constructors have won the most races.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.constructor_performance AS
SELECT
    q.Constructor_Name,
    r.Season,
    COUNT(CASE WHEN r.Position = 1 THEN 1 END) AS wins,
    COUNT(CASE WHEN r.Position <= 3 THEN 1 END) AS podiums,
    COUNT(*) AS total_races,
    ROUND((COUNT(CASE WHEN r.Position = 1 THEN 1 END) / COUNT(*)) * 100, 2) AS win_percentage,
    ROUND((COUNT(CASE WHEN r.Position <= 3 THEN 1 END) / COUNT(*)) * 100, 2) AS podium_percentage
FROM saffatandsourik.formula1_int.Race_Results r
JOIN saffatandsourik.formula1_int.Qualifying_Results q
    ON r.Driver_ID = q.Driver_ID
    AND r.Season = q.Year
    AND r.Round = q.Round
GROUP BY q.Constructor_Name, r.Season
ORDER BY r.Season DESC, wins DESC;


Query is running:   0%|          |

""


## 3.What is the average number of pit stops per driver per race?
###This question examines pit stop frequency and duration.

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.lap_frequency AS
SELECT
    p.driver_id,
    d.first_name,
    d.last_name,
    COUNT(*) AS total_pit_stops,
    ROUND(AVG(p.pit_duration), 2) AS avg_pit_duration
FROM saffatandsourik.formula1_int.Pit p
JOIN saffatandsourik.formula1_int.Drivers d
    ON p.driver_id = d.driver_id
GROUP BY p.driver_id, d.first_name, d.last_name
ORDER BY total_pit_stops DESC;


Query is running:   0%|          |

""


## 4. Which drivers consistently perform well across seasons, and how do their career stats compare?
### This question helps track driver and constructor consistency in achieving high finishes.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.consistency AS
SELECT
    r.Driver_ID,
    d.First_Name,
    d.Last_Name,
    q.Constructor_Name,
    r.Season,
    COUNT(*) AS races_participated,
    ROUND(AVG(r.Position), 2) AS avg_finish_position,
    SUM(r.Points) AS total_points,
    COUNT(CASE WHEN r.Position = 1 THEN 1 END) AS career_wins,
    COUNT(CASE WHEN r.Position <= 3 THEN 1 END) AS career_podiums,
    COUNT(CASE WHEN r.Status NOT IN ('Finished', 'Running') THEN 1 END) AS dnfs,
    ROUND((COUNT(CASE WHEN r.Status NOT IN ('Finished', 'Running') THEN 1 END) / COUNT(*)) * 100, 2) AS dnf_rate
FROM saffatandsourik.formula1_int.Race_Results r
JOIN saffatandsourik.formula1_int.Drivers d
    ON r.Driver_ID = d.Driver_ID
JOIN saffatandsourik.formula1_int.Qualifying_Results q
    ON r.Driver_ID = q.Driver_ID
    AND r.Season = q.Year
GROUP BY r.Driver_ID, d.First_Name, d.Last_Name, q.Constructor_Name, r.Season
ORDER BY r.Season DESC, total_points DESC;


Query is running:   0%|          |

""


## 5. Which constructors have the most reliable cars, and how does reliability impact their race performance?
### Analyzing car reliability helps teams improve their engineering and avoid mechanical failures.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.reliability AS
SELECT
    c.Car,
    c.Constructor_Reliability,
    COUNT(r.Race_ID) AS total_races,
    COUNT(CASE WHEN r.Status NOT IN ('Finished', 'Running') THEN 1 END) AS failures,
    ROUND((COUNT(CASE WHEN r.Status NOT IN ('Finished', 'Running') THEN 1 END) / COUNT(r.Race_ID)) * 100, 2) AS failure_rate,
    COUNT(CASE WHEN r.Status LIKE '%Engine%' THEN 1 END) AS engine_failures,
    COUNT(CASE WHEN r.Status LIKE '%Gearbox%' THEN 1 END) AS gearbox_failures,
    COUNT(CASE WHEN r.Status LIKE '%Brakes%' THEN 1 END) AS brake_failures,
    SUM(r.Points) AS total_points,
    COUNT(CASE WHEN r.Position = 1 THEN 1 END) AS wins,
    COUNT(CASE WHEN r.Position <= 3 THEN 1 END) AS podium_finishes
FROM saffatandsourik.formula1_int.Car_Info c
JOIN saffatandsourik.formula1_int.Race_Results r
    ON c.Driver_ID = r.Driver_ID
    AND c.Year = r.Season
GROUP BY c.Car, c.Constructor_Reliability
ORDER BY failure_rate DESC, total_points DESC;


Query is running:   0%|          |

""


## 6. How valuable is pole position at different circuits, and how often does it lead to a podium finish?
### This examines whether starting in P1 (pole position) translates to a race victory.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.pole_position AS
SELECT
    r.Circuit_ID,
    c.Circuit_Name,
    COUNT(q.Position) AS times_started_pole,
    COUNT(CASE WHEN rr.Position = 1 THEN 1 END) AS times_won,
    COUNT(CASE WHEN rr.Position <= 3 THEN 1 END) AS times_on_podium,
    ROUND((COUNT(CASE WHEN rr.Position = 1 THEN 1 END) / COUNT(q.Position)) * 100, 2) AS pole_to_win_conversion_rate,
    ROUND((COUNT(CASE WHEN rr.Position <= 3 THEN 1 END) / COUNT(q.Position)) * 100, 2) AS pole_to_podium_conversion_rate,
    ROUND(AVG(q.Position - rr.Position), 2) AS avg_positions_lost_from_pole
FROM saffatandsourik.formula1_int.Qualifying_Results q
JOIN saffatandsourik.formula1_int.Races r
    ON q.Year = r.Year AND q.Round = r.Round
JOIN saffatandsourik.formula1_int.Race_Results rr
    ON q.Driver_ID = rr.Driver_ID AND q.Year = rr.Season AND q.Round = rr.Round
JOIN saffatandsourik.formula1_int.Circuits c
    ON r.Circuit_ID = c.Circuit_ID
WHERE q.Position = 1
GROUP BY r.Circuit_ID, c.Circuit_Name
ORDER BY pole_to_win_conversion_rate DESC;


Query is running:   0%|          |

""


## 7 Which circuits have the most retirements, and what are the most common reasons?


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.retirements AS
SELECT
    c.Circuit_Name,
    COUNT(*) AS retirements,
    COUNT(CASE WHEN rr.Status LIKE '%Collision%' THEN 1 END) AS collision_retirements,
    COUNT(CASE WHEN rr.Status LIKE '%Mechanical%' THEN 1 END) AS mechanical_failures,
    COUNT(CASE WHEN rr.Status LIKE '%Engine%' THEN 1 END) AS engine_failures,
    COUNT(CASE WHEN rr.Status LIKE '%Gearbox%' THEN 1 END) AS gearbox_failures,
    COUNT(CASE WHEN rr.Status LIKE '%Brakes%' THEN 1 END) AS brake_failures
FROM saffatandsourik.formula1_int.Race_Results rr
JOIN saffatandsourik.formula1_int.Races r
    ON rr.Race_ID = r.Race_ID
JOIN saffatandsourik.formula1_int.Circuits c
    ON r.Circuit_ID = c.Circuit_ID
WHERE rr.Status NOT IN ('Finished', 'Running')
GROUP BY c.Circuit_Name
ORDER BY retirements DESC
LIMIT 10;


Query is running:   0%|          |

""


## 8. Which teams, drivers, and circuits experience the most race retirements, and what are the most common reasons?
### This identifies the most frequent reasons drivers fail to finish.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.finish_fail AS
SELECT
    rr.Status,
    COUNT(*) AS occurrences,
    q.Constructor_Name,
    d.First_Name,
    d.Last_Name,
    ci.Constructor_Reliability,
    r.Circuit_ID,
    cir.Circuit_Name
FROM saffatandsourik.formula1_int.Race_Results rr
JOIN saffatandsourik.formula1_int.Qualifying_Results q
    ON rr.Driver_ID = q.Driver_ID
    AND rr.Season = q.Year
JOIN saffatandsourik.formula1_int.Drivers d
    ON rr.Driver_ID = d.Driver_ID
JOIN saffatandsourik.formula1_int.Car_Info ci
    ON rr.Driver_ID = ci.Driver_ID
    AND rr.Season = ci.Year
JOIN saffatandsourik.formula1_int.Races r
    ON rr.Race_ID = r.Race_ID
JOIN saffatandsourik.formula1_int.Circuits cir
    ON r.Circuit_ID = cir.Circuit_ID
WHERE rr.Status NOT IN ('Finished', 'Running')
GROUP BY rr.Status, q.Constructor_Name, d.First_Name, d.Last_Name, ci.Constructor_Reliability, r.Circuit_ID, cir.Circuit_Name
ORDER BY occurrences DESC;


Query is running:   0%|          |

""


## 9. Which constructors perform the best each season, based on race results, wins, podiums, and reliability?
### Tracks seasonal trends for each constructor.


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.best_constructor AS
SELECT
    q.Constructor_Name,
    r.Season,
    COUNT(*) AS races_participated,
    SUM(r.Points) AS total_points,
    ROUND(AVG(r.Position), 2) AS avg_finish_position,
    COUNT(CASE WHEN r.Position = 1 THEN 1 END) AS wins,
    COUNT(CASE WHEN r.Position <= 3 THEN 1 END) AS podiums,
    COUNT(CASE WHEN r.Status NOT IN ('Finished', 'Running') THEN 1 END) AS dnfs,  -- Did Not Finish
    ROUND((COUNT(CASE WHEN r.Status NOT IN ('Finished', 'Running') THEN 1 END) / COUNT(*)) * 100, 2) AS dnf_rate
FROM saffatandsourik.formula1_int.Race_Results r
JOIN saffatandsourik.formula1_int.Qualifying_Results q
    ON r.Driver_ID = q.Driver_ID
    AND r.Season = q.Year
GROUP BY q.Constructor_Name, r.Season
ORDER BY r.Season DESC, total_points DESC;


Query is running:   0%|          |

""


## 10. Which drivers consistently improve from their qualifying position to their race result, and how often do they gain positions?
Analyzes who gains the most positions after qualifying.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.consistent_drivers AS
SELECT
    q.Driver_ID,
    d.First_Name,
    d.Last_Name,
    COUNT(*) AS total_races,
    COUNT(CASE WHEN (q.Position - r.Position) > 0 THEN 1 END) AS races_improved,
    COUNT(CASE WHEN (q.Position - r.Position) < 0 THEN 1 END) AS races_lost_positions,
    ROUND(AVG(q.Position - r.Position), 2) AS avg_positions_gained,
    COUNT(CASE WHEN r.Position = 1 THEN 1 END) AS wins,
    COUNT(CASE WHEN r.Position <= 3 THEN 1 END) AS podium_finishes
FROM saffatandsourik.formula1_int.Qualifying_Results q
JOIN saffatandsourik.formula1_int.Race_Results r
    ON q.Driver_ID = r.Driver_ID
    AND q.Year = r.Season
    AND q.Round = r.Round
JOIN saffatandsourik.formula1_int.Drivers d
    ON q.Driver_ID = d.Driver_ID
GROUP BY q.Driver_ID, d.First_Name, d.Last_Name
ORDER BY avg_positions_gained DESC, wins DESC;


Query is running:   0%|          |

""


## 11.  Which constructors have the most reliable cars and how does reliability impact their race performance?
### Reliability matters in long seasons—this shows who finishes most races.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.constructor_reliable AS
SELECT
    q.Constructor_Name,
    COUNT(r.Race_ID) AS total_races,
    COUNT(CASE WHEN r.Status = 'Finished' THEN 1 END) AS completed_races,
    ROUND((COUNT(CASE WHEN r.Status = 'Finished' THEN 1 END) / COUNT(r.Race_ID)) * 100, 2) AS finish_rate
FROM saffatandsourik.formula1_int.Race_Results r
JOIN saffatandsourik.formula1_int.Qualifying_Results q
    ON r.Driver_ID = q.Driver_ID
    AND r.Season = q.Year
GROUP BY q.Constructor_Name
ORDER BY finish_rate DESC;


Query is running:   0%|          |

""


## 12. Which drivers have demonstrated the most consistent performance over their careers, factoring in race results, qualifying positions, and car performance?
### More experienced drivers may perform better—this query examines trends

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE saffatandsourik.formula1_mrt.consistent_performance AS
SELECT
    d.Driver_ID,
    d.First_Name,
    d.Last_Name,
    COUNT(DISTINCT r.Season) AS seasons_raced,
    SUM(r.Points) AS total_points,
    ROUND(AVG(r.Position), 2) AS avg_race_finish,
    ROUND(AVG(q.Position), 2) AS avg_qualifying_position,
    COUNT(CASE WHEN r.Position = 1 THEN 1 END) AS career_wins,
    COUNT(CASE WHEN r.Position <= 3 THEN 1 END) AS career_podiums,
    AVG(c.Car_Competitiveness) AS avg_car_competitiveness
FROM saffatandsourik.formula1_int.Race_Results r
JOIN saffatandsourik.formula1_int.Drivers d
    ON r.Driver_ID = d.Driver_ID
JOIN saffatandsourik.formula1_int.Qualifying_Results q
    ON r.Driver_ID = q.Driver_ID
    AND r.Season = q.Year
JOIN saffatandsourik.formula1_int.Car_Info c
    ON r.Driver_ID = c.Driver_ID
    AND r.Season = c.Year
GROUP BY d.Driver_ID, d.First_Name, d.Last_Name
ORDER BY total_points DESC, career_wins DESC;


Query is running:   0%|          |

""
